<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Push Swap método "Modern Times"
En honor a la película de 1936 de Chaplin.

In [1]:
# FUNCIONES

def lista_ordenada(lis, orden_inverso=False): # la función da True si ponemos lista_ordenada([4,3,2,1], orden_inverso=True)
    ordenada = False
    aux = lis[:]
    aux.sort(reverse=orden_inverso)
    if aux == lis:
        ordenada = True
    return ordenada

def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

In [2]:
# Generación de la pila a
from random import sample, seed
seed()
n = 500   # número de elementos de la pila
#a = sample(range(1, n+1), n)
a = [175, 38, 151, 167, 17, 168, 26, 244, 229, 364, 137, 54, 55, 245, 403, 319, 402, 67, 51, 479, 362, 224, 482, 63, 237, 277, 268, 201, 335, 490, 239, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 261, 27, 376, 144, 337, 248, 363, 117, 70, 193, 200, 84, 284, 108, 410, 438, 81, 177, 104, 236, 174, 264, 145, 316, 430, 71, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 266, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 153, 303, 115, 445, 269, 196, 77, 199, 208, 333, 166, 259, 413, 344, 468, 176, 95, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 110, 179, 453, 123, 442, 188, 72, 101, 475, 477, 99, 427, 447, 382, 270, 96, 407, 255, 191, 408, 133, 273, 370, 343, 112, 74, 467, 56, 149, 249, 73, 189, 138, 234, 367, 198, 212, 279, 136, 124, 491, 478, 82, 496, 287, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 290, 474, 247, 242, 298, 296, 473, 352, 397, 223, 65, 360, 495, 230, 369, 465, 97, 69, 422, 492, 122, 454, 155, 321, 353, 121, 211, 357, 21, 165, 30, 336, 318, 171, 271, 254, 451, 143, 295, 131, 500, 233, 358, 16, 449, 231, 156, 240, 493, 466, 172, 164, 405, 263, 160, 253, 202, 476, 415, 302, 210, 464, 359, 23, 481, 332, 11, 347, 39, 418, 361, 260, 246, 42, 119, 440, 10, 44, 341, 293, 94, 312, 14, 85, 299, 68, 238, 256, 301, 354, 472, 107, 486, 157, 148, 125, 329, 424, 89, 311, 380, 36, 431, 154, 215, 393, 265, 109, 334, 102, 228, 159, 22, 52, 126, 459, 75, 272, 275, 432, 116, 141, 373, 135, 60, 294, 326, 426, 53, 5, 134, 297, 29, 267, 289, 389, 18, 391, 460, 484, 340, 180, 111, 195, 173, 470, 128, 92, 207, 366, 204, 350, 441, 58, 182, 120, 25, 377, 130, 147, 411, 163, 384, 305, 281, 190, 43, 222, 375, 203, 142, 300, 93, 320, 497, 439, 194, 338, 184, 140, 308, 12, 241, 323, 216, 150, 197, 456, 258, 88, 31, 409, 420, 435, 15, 219, 100, 317, 461, 429, 419, 3, 187, 46, 414, 66, 178, 444, 226, 416, 252, 87, 383, 181, 443, 345, 83, 342, 80, 62, 90, 385, 457, 448, 483, 49, 485, 50, 35, 152, 315, 386, 161, 251, 325, 227, 205, 45, 313, 185, 114, 446, 371, 330, 437, 37, 452, 327, 186, 76, 158, 368, 129, 118, 292, 232, 218, 398, 396, 423, 170, 314, 421, 6, 488, 235, 127, 286, 480, 433, 47, 28, 250, 322, 276, 489, 394, 349, 436, 33, 217, 463, 306, 471, 450, 404, 278, 225, 339, 399, 355, 351, 213, 9, 285, 98, 458, 146, 113, 103, 328, 221, 78, 91, 499, 220, 4, 498, 390, 388, 40, 412, 348, 378, 41, 288, 86, 7, 283, 192, 395, 105]
#a = [220, 4, 498, 390, 388, 40, 412, 348, 378, 41, 288, 86, 7, 283, 192, 395, 105, 175, 38, 151, 167, 168, 244, 229, 364, 137, 245, 403, 319, 402, 51, 479, 362, 224, 482, 63, 237, 277, 268, 201, 335, 490, 239, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 261, 27, 376, 144, 337, 248, 363, 117, 193, 200, 284, 410, 438, 81, 177, 104, 236, 174, 264, 316, 430, 71, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 266, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 303, 115, 445, 269, 196, 77, 199, 208, 333, 259, 413, 344, 468, 95, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 110, 453, 123, 442, 72, 101, 475, 477, 99, 427, 447, 382, 270, 96, 407, 255, 408, 133, 273, 370, 343, 112, 74, 467, 56, 149, 249, 73, 189, 138, 234, 367, 279, 136, 124, 491, 478, 82, 496, 287, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 290, 474, 247, 242, 298, 296, 473, 352, 397, 65, 360, 495, 369, 465, 97, 69, 422, 492, 122, 454, 155, 321, 353, 121, 211, 357, 21, 165, 30, 336, 318, 171, 271, 254, 451, 143, 295, 131, 500, 358, 16, 449, 231, 156, 493, 466, 172, 164, 405, 263, 160, 202, 476, 415, 302, 210, 464, 359, 23, 481, 332, 11, 347, 39, 418, 361, 246, 42, 119, 440, 10, 44, 341, 94, 312, 14, 85, 68, 238, 256, 354, 472, 107, 486, 157, 148, 125, 424, 89, 311, 380, 36, 431, 154, 215, 393, 265, 109, 102, 228, 159, 22, 52, 126, 459, 75, 272, 275, 432, 116, 141, 373, 135, 60, 294, 326, 426, 53, 5, 134, 297, 29, 267, 289, 389, 18, 391, 460, 484, 180, 111, 195, 173, 470, 128, 92, 207, 204, 350, 441, 58, 182, 120, 25, 130, 147, 411, 163, 305, 281, 190, 43, 222, 375, 203, 142, 300, 93, 320, 497, 439, 194, 338, 184, 140, 308, 12, 241, 323, 216, 150, 197, 456, 258, 88, 31, 15, 219, 100, 317, 461, 429, 419, 3, 187, 46, 414, 66, 178, 226, 416, 252, 87, 383, 181, 443, 345, 83, 342, 80, 62, 90, 385, 448, 49, 50, 35, 152, 315, 386, 161, 251, 325, 227, 205, 45, 313, 185, 114, 446, 371, 330, 437, 37, 452, 327, 186, 76, 158, 368, 129, 118, 292, 232, 218, 398, 396, 423, 170, 314, 421, 6, 235, 127, 286, 480, 433, 47, 28, 250, 322, 276, 394, 349, 436, 33, 217, 463, 306, 471, 450, 404, 278, 225, 339, 399, 355, 351, 213, 9, 285, 98, 458, 146, 113, 103, 328, 221, 78, 91]
#a = [450, 404, 278, 225, 339, 399, 355, 351, 213, 9, 285, 98, 458, 146, 113, 103, 328, 221, 78, 91, 220, 498, 390, 388, 412, 348, 378, 288, 7, 283, 192, 395, 175, 38, 244, 364, 137, 245, 403, 319, 402, 51, 479, 362, 224, 482, 63, 277, 268, 201, 335, 490, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 27, 376, 144, 337, 248, 363, 117, 193, 200, 284, 410, 438, 81, 177, 104, 236, 174, 316, 430, 71, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 303, 115, 445, 196, 77, 199, 208, 333, 259, 413, 344, 468, 95, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 110, 453, 123, 442, 72, 101, 475, 477, 99, 427, 447, 382, 96, 407, 255, 408, 133, 370, 343, 112, 74, 467, 56, 149, 249, 73, 189, 138, 234, 367, 136, 124, 491, 478, 82, 496, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 474, 247, 242, 296, 473, 397, 65, 360, 495, 369, 465, 97, 69, 422, 492, 122, 454, 155, 321, 121, 211, 21, 165, 30, 336, 318, 171, 271, 254, 451, 143, 295, 131, 500, 16, 449, 231, 156, 493, 466, 172, 164, 405, 263, 160, 202, 476, 415, 302, 210, 464, 23, 481, 332, 11, 347, 39, 418, 246, 42, 119, 440, 10, 44, 341, 94, 312, 14, 85, 68, 238, 256, 354, 472, 107, 486, 157, 148, 125, 424, 89, 311, 36, 431, 154, 215, 393, 265, 109, 102, 228, 159, 22, 52, 126, 459, 75, 272, 275, 432, 116, 141, 373, 135, 60, 294, 326, 426, 53, 5, 134, 297, 29, 267, 289, 18, 460, 484, 180, 111, 195, 173, 470, 128, 92, 207, 204, 350, 441, 58, 182, 120, 25, 130, 147, 163, 305, 281, 190, 43, 222, 375, 203, 142, 300, 93, 320, 497, 439, 194, 338, 184, 140, 308, 12, 241, 323, 216, 150, 197, 456, 258, 88, 31, 15, 219, 100, 317, 461, 429, 419, 3, 187, 46, 66, 178, 226, 252, 87, 383, 181, 345, 83, 342, 80, 62, 90, 385, 49, 50, 35, 152, 315, 386, 161, 251, 325, 227, 205, 45, 313, 185, 114, 446, 371, 330, 437, 37, 327, 186, 76, 158, 368, 129, 118, 292, 232, 218, 398, 396, 423, 170, 314, 421, 6, 235, 127, 286, 480, 433, 47, 28, 250, 322, 276, 394, 349, 436, 33, 217, 306]
#a = [33, 217, 306, 450, 404, 278, 225, 339, 399, 355, 351, 213, 285, 98, 458, 146, 113, 103, 328, 221, 78, 91, 220, 498, 390, 388, 412, 348, 378, 288, 7, 283, 192, 395, 175, 244, 364, 137, 245, 403, 319, 402, 479, 362, 224, 482, 277, 268, 201, 335, 490, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 27, 376, 144, 337, 248, 363, 117, 193, 200, 284, 410, 438, 81, 177, 104, 236, 174, 316, 430, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 303, 115, 445, 196, 199, 208, 333, 259, 413, 344, 468, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 453, 442, 72, 101, 475, 477, 99, 427, 447, 382, 96, 407, 255, 408, 370, 343, 112, 74, 467, 56, 249, 73, 189, 138, 234, 367, 136, 124, 491, 478, 82, 496, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 474, 247, 242, 296, 473, 397, 65, 360, 495, 369, 465, 97, 69, 422, 492, 122, 454, 321, 121, 211, 21, 30, 336, 318, 271, 254, 451, 143, 295, 131, 500, 16, 449, 231, 156, 493, 466, 164, 405, 263, 160, 476, 415, 302, 464, 23, 481, 332, 11, 347, 39, 418, 42, 119, 440, 10, 44, 341, 94, 312, 14, 85, 68, 238, 354, 472, 107, 486, 157, 148, 125, 424, 89, 311, 36, 431, 154, 215, 393, 109, 102, 228, 159, 22, 52, 126, 459, 75, 432, 116, 141, 373, 135, 60, 326, 426, 53, 5, 134, 29, 267, 289, 18, 460, 484, 180, 111, 195, 173, 470, 128, 92, 207, 204, 350, 441, 58, 182, 120, 25, 130, 147, 163, 281, 190, 43, 222, 375, 203, 142, 300, 93, 497, 439, 194, 184, 140, 308, 12, 241, 323, 216, 150, 197, 456, 258, 88, 31, 15, 219, 100, 317, 461, 429, 419, 3, 187, 46, 66, 178, 226, 252, 87, 181, 345, 83, 342, 80, 62, 90, 49, 50, 35, 152, 315, 161, 251, 325, 227, 205, 45, 313, 185, 114, 446, 371, 330, 437, 37, 327, 186, 76, 158, 368, 129, 118, 292, 232, 218, 396, 170, 314, 421, 6, 235, 127, 286, 480, 47, 28, 250, 322, 276, 394, 349]
print('a:', a)
b = []
a_original = a[:]

a: [175, 38, 151, 167, 17, 168, 26, 244, 229, 364, 137, 54, 55, 245, 403, 319, 402, 67, 51, 479, 362, 224, 482, 63, 237, 277, 268, 201, 335, 490, 239, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 261, 27, 376, 144, 337, 248, 363, 117, 70, 193, 200, 84, 284, 108, 410, 438, 81, 177, 104, 236, 174, 264, 145, 316, 430, 71, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 266, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 153, 303, 115, 445, 269, 196, 77, 199, 208, 333, 166, 259, 413, 344, 468, 176, 95, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 110, 179, 453, 123, 442, 188, 72, 101, 475, 477, 99, 427, 447, 382, 270, 96, 407, 255, 191, 408, 133, 273, 370, 343, 112, 74, 467, 56, 149, 249, 73, 189, 138, 234, 367, 198, 212, 279, 136, 124, 491, 478, 82, 496, 287, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 290, 474, 247, 242, 298, 296, 473, 352, 397, 223, 65, 360, 495, 230, 369, 465, 97, 69, 422, 492, 122, 454, 155, 321, 353, 121, 211, 357, 21, 165, 

In [3]:
def algoritmo_LIS(arr):
    n = len(arr)
    lis = [1]*n
    prev = list(range(n))  # una lista con los index
    
    # Compute optimized LIS values in bottom up manner
    for i in range (1, n):
        for j in range(i):
            if arr[i] > arr[j] and lis[i] < lis[j] + 1:
                lis[i] = lis[j]+1
                prev[i] = j

    # Initialize maximum to 0 to get the maximum of all
    # LIS
    maximum = 0
    idx = 0

    # Pick maximum of all LIS values
    for i in range(n):
        if maximum < lis[i]:
            maximum = lis[i]
            idx = i

    seq = [arr[idx]]
    while idx != prev[idx]:
        idx = prev[idx]
        seq.append(arr[idx])

    return (maximum, list(reversed(seq)))

(nlis, lis) = algoritmo_LIS(a)   #nlist es el número de elementos de la lista lis

In [4]:
# Llevamos de la pila A a la B los elementos de la LIS
contador = 0
for i in range(nlis):
    while a_original[contador] != lis[i]:
        ra(a,b); contador += 1; 
    else:
        pb(a,b); contador += 1
print(a_original)
print(lis)
print(a)
print(b)
print("contador:", contador)

[175, 38, 151, 167, 17, 168, 26, 244, 229, 364, 137, 54, 55, 245, 403, 319, 402, 67, 51, 479, 362, 224, 482, 63, 237, 277, 268, 201, 335, 490, 239, 307, 324, 387, 34, 274, 280, 365, 162, 372, 214, 261, 27, 376, 144, 337, 248, 363, 117, 70, 193, 200, 84, 284, 108, 410, 438, 81, 177, 104, 236, 174, 264, 145, 316, 430, 71, 487, 243, 425, 19, 183, 417, 57, 406, 282, 169, 392, 266, 20, 455, 356, 291, 209, 462, 48, 346, 257, 309, 64, 59, 32, 2, 153, 303, 115, 445, 269, 196, 77, 199, 208, 333, 166, 259, 413, 344, 468, 176, 95, 400, 8, 206, 469, 381, 310, 304, 139, 379, 132, 1, 428, 110, 179, 453, 123, 442, 188, 72, 101, 475, 477, 99, 427, 447, 382, 270, 96, 407, 255, 191, 408, 133, 273, 370, 343, 112, 74, 467, 56, 149, 249, 73, 189, 138, 234, 367, 198, 212, 279, 136, 124, 491, 478, 82, 496, 287, 262, 331, 434, 79, 24, 494, 374, 106, 61, 13, 401, 290, 474, 247, 242, 298, 296, 473, 352, 397, 223, 65, 360, 495, 230, 369, 465, 97, 69, 422, 492, 122, 454, 155, 321, 353, 121, 211, 357, 21, 165, 30,

In [ ]:
def pasos_estimados(a, b, index_a):
    # estima el número de pasos para colocar el elemento de la pila A cuyo index_a nos pasan, en el sitio adecuado de la pila B
    # las pilas A y B que recibimos como parámetros no se han de alterar por parte de esta función
    


    conta2 = 0
    while not lista_ordenada(a):   # se repite hasta que A quede ordenada. Si solo queda un elemento en A es que está ordenada
        while min(a) != a[0]:   # buscamos el mínimo y vamos haciendo ra o rra hasta que quede el primero
            if a.index(min(a)) <= int(len(b)/2):
                ra(a,b)
            else:
                rra(a,b)
            conta2 += 1
        if not lista_ordenada(a):
            pb(a,b); conta2 += 1






    




    return pasos


def busca_candidato(a, b):
    candidato = 0         # determinar el elemento de la pila A que menos pasos necesita para ubicarse en su sitio de la pila B
    pasos_min = 2**31-1   # inicializamos la variable con infinito o un número de pasos que sabemos que es muy muy grande 2**31-1 == 2147483647

    for index_a in range(len(a)):
        p = pasos(a, b, index_a)
        if p < pasos_min:
            pasos_min = p
            candidato = a[index_a]   #ahora el nuevo candidato es el elemento de A que requiere, de momento, menos pasos para ir a su sitio en la pila B
    return candidato




In [24]:
elemento = 6
b = [20, 15, 12, 10, 5, 2]

dif = 2**31-1  #inicializamos la diferencia con infinito o casi
index_insercion = 0   #inicializamos el index de B donde se realizará la inserción del elemento
for i,v in enumerate(b):
    if abs(v-elemento) < dif:
        dif = abs(v-elemento)
        indice_insercion = i

print(indice_insercion)

# convertirmos el indice_insercion en pasos necesarios para el elemento considerado

mitad = int(len(b)/2)

print("mitad:", mitad)

if elemento < b[mitad]:
    print("signo negativo")
else:
    print("signo positivo")


#    if a.index(min(a)) <= int(len(b)/2):




4
mitad: 3
signo negativo
